### 데이터 가져오기

In [5]:
file = '../DATA/2023년 연료원별 시간대별 설비용량 및 전력거래량 (1).xlsx'
import pandas as pd

df = pd.read_excel(file, sheet_name='Sheet1', skiprows=5)
print(df.head())

       거래일자  시간  연료원  설비용량(MW)  전력거래량(MWh)
0  20230101   1  원자력     24650       19561
1  20230101   2  원자력     24650       19561
2  20230101   3  원자력     24650       19562
3  20230101   4  원자력     24650       19560
4  20230101   5  원자력     24650       19559


In [ ]:
# %pip install openpyxl

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179208 entries, 0 to 179207
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   거래일자        179208 non-null  int64 
 1   시간          179208 non-null  int64 
 2   연료원         179208 non-null  object
 3   설비용량(MW)    179208 non-null  int64 
 4   전력거래량(MWh)  179208 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 6.8+ MB
